In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# reading input data
with open("../data/tinyshakespeare.txt", "r", encoding="utf-8") as file:
    data = file.read()

In [3]:
print(f"Length of dataset w.r.t chars: {len(data)}\n")
print(data[:1000])

Length of dataset w.r.t chars: 1115394

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger

In [4]:
# all unique chars
chars = sorted(list(set(data)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# tokenization (mapping for char to int)
ch_to_id = {ch:i for i, ch in enumerate(chars)}
id_to_ch = {i:ch for i, ch in enumerate(chars)}
def encode(s):
    return [ch_to_id[c] for c in s] # encoder: take a string, output a list of integers
def decode(l):
    return ''.join([id_to_ch[i] for i in l]) # decoder: take a list of integers, output a string

# tests
print(encode("Hello world!"))
print(decode(encode("Hello World!")))

[20, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 2]
Hello World!


In [6]:
# tokenizing enitire dataset and converting them into tensors
data = torch.tensor(encode(data), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # 1st 1000 char tensors

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
# train, val split
thresh = int(0.9 * len(data))
train_data = data[:thresh] # 90%
val_data = data[thresh:] # 10%

In [8]:
# dataloader: outs batches of x and y pairs
batch_size = 4 # n sequence to process parallaly
block_size = 8 # context length

def get_batches(split):
    """
    generate batched in and out pairs
    """
    data = train_data if split == "train" else val_data
    idx = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i + block_size] for i in idx])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in idx])
    return x, y

mini_x, mini_y = get_batches("train") # batch
print(f"Inputs shape: {mini_x.shape}\nInputs: {mini_x}\n")
print(f"Target shape: {mini_y.shape}\nTarget: {mini_y}\n")
print("--------------------------------------------------\n")

# insights into the batched data
for b in range(batch_size):
    for t in range(block_size):
        context = mini_x[b, :t+1]
        target = mini_y[b, t]
        print(f"Input: {context.tolist()}, Target: {target}")

Inputs shape: torch.Size([4, 8])
Inputs: tensor([[ 0,  5, 32, 47, 57,  1, 52, 53],
        [27, 56,  1, 40, 43, 52, 42,  1],
        [43,  1, 24, 53, 56, 42, 10,  0],
        [46, 43, 56,  8,  0,  0, 14, 53]])

Target shape: torch.Size([4, 8])
Target: tensor([[ 5, 32, 47, 57,  1, 52, 53, 58],
        [56,  1, 40, 43, 52, 42,  1, 53],
        [ 1, 24, 53, 56, 42, 10,  0, 18],
        [43, 56,  8,  0,  0, 14, 53, 58]])

--------------------------------------------------

Input: [0], Target: 5
Input: [0, 5], Target: 32
Input: [0, 5, 32], Target: 47
Input: [0, 5, 32, 47], Target: 57
Input: [0, 5, 32, 47, 57], Target: 1
Input: [0, 5, 32, 47, 57, 1], Target: 52
Input: [0, 5, 32, 47, 57, 1, 52], Target: 53
Input: [0, 5, 32, 47, 57, 1, 52, 53], Target: 58
Input: [27], Target: 56
Input: [27, 56], Target: 1
Input: [27, 56, 1], Target: 40
Input: [27, 56, 1, 40], Target: 43
Input: [27, 56, 1, 40, 43], Target: 52
Input: [27, 56, 1, 40, 43, 52], Target: 42
Input: [27, 56, 1, 40, 43, 52, 42], Target:

In [18]:
# bigram class
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers batch, time/context
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
    def sample(self, idx, max_new_tokens):
        return decode(self.generate(idx, max_new_tokens)[0].tolist())
    
bigram_model = BigramLanguageModel(vocab_size)
logits, loss = bigram_model(mini_x, mini_y)
print(f"Loss: {loss}")

Loss: 4.807022571563721


In [45]:
# prediction before training
print(bigram_model.sample(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100))


QSxVTLCvqSNeN?'j
keCNY-T
dteYGAeW&f$Jx;z3UBSeAHsL&
KuIZkIp$,cLLfHVT,'ZbW.TI:ukL.HZpSbwPSCyifaEdg!.HH


In [ ]:
# # predictions before training
# print(decode(bigram_model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

In [46]:
# optimizer
optimizer = torch.optim.AdamW(bigram_model.parameters(), lr=1e-3)

In [47]:
# Training bigram model
batch_size = 32
epochs = 10000

for epoch in range(epochs):
    # getting batches
    xb, yb = get_batches("train")
    
    # loss
    logits, loss = bigram_model(xb, yb)
    
    # setting grad to zero
    optimizer.zero_grad(set_to_none=True)
    
    # backprop
    loss.backward()
    
    # update params
    optimizer.step()
    if (epoch+1) % 1000 == 0:
        print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1000, Loss: 3.8054986000061035
Epoch: 2000, Loss: 3.324122667312622
Epoch: 3000, Loss: 2.8780388832092285
Epoch: 4000, Loss: 2.6557414531707764
Epoch: 5000, Loss: 2.6466286182403564
Epoch: 6000, Loss: 2.3895211219787598
Epoch: 7000, Loss: 2.5135180950164795
Epoch: 8000, Loss: 2.5664174556732178
Epoch: 9000, Loss: 2.5909507274627686
Epoch: 10000, Loss: 2.3946382999420166


In [51]:
# prediction after training
print(bigram_model.sample(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500))


Angty, host wiccerth ir wndeanstJou bur dy sons KI in, l all alertstud tt
IN:
Whaibeeasor wado s;
FO fthenckithigherevey thooures;
veionnd hece hel agh e o, ldurero gle t h f secousengharsar.


O:
LOUt t infercut phe be ness;
TXger.
Wayofay p,
Hal torisou hmand ms athondlouelee w'TI t s.

WA:
HAn ird sea pravear ur,
Aco rinden.
Ound leOF:
Whe, it
Thais id!
s st bouce sisilmaknorrsad ome Gondss:
MEve I knomelx homurete ISEThe;
Mans, yowe ayoure chinof hearerathean, slavin f ie tof is Thre hey rin


In [ ]:
# # predictions after training
# print(decode(bigram_model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

In [57]:
# intuition for masked self-attention
B, T, C = 4, 8, 2 # Batch, Time(context length), Channels(embeddings or info at B,T)
x = torch.randn(B, T, C)
print(f"x Shape: {x.shape}")

# we want ith token to have attention to all tokens from i-T to i
# hence x[b, t] = mean of i<t x[b, i]
xbow = torch.zeros((B, T, C)) # averaging
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)
x[0], xbow[0]

x Shape: torch.Size([4, 8, 2])


(tensor([[ 0.5278,  0.8184],
         [-0.0731,  0.0911],
         [-0.2688,  0.6280],
         [ 0.5763, -0.3307],
         [ 1.2834,  0.6111],
         [ 0.3370, -1.1037],
         [-0.4783, -0.1251],
         [-0.5722, -2.0273]]),
 tensor([[ 0.5278,  0.8184],
         [ 0.2274,  0.4547],
         [ 0.0620,  0.5125],
         [ 0.1906,  0.3017],
         [ 0.4091,  0.3636],
         [ 0.3971,  0.1190],
         [ 0.2721,  0.0842],
         [ 0.1665, -0.1798]]))

In [65]:
# masked self-attention using matrix multiplication
weights = torch.tril(torch.ones(T, T)) # lower triangular matrix
weights = weights / weights.sum(1, keepdim=True) # row normalization
print("Weights for self-attention: \n", weights)
xbow2 =  weights @ x # (B, T, T) @ (B, T, C) = (B, T, C), here the weights are averaging the context so that the ith token has attention to all tokens from i-T to i
torch.allclose(xbow, xbow2) # tells if the two tensors are equal

Weights for self-attention: 
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


True

In [66]:
# masked self-attention using softmax
tril = torch.tril(torch.ones(T, T)) # lower triangular matrix
weights = torch.zeros((T, T))
weights = weights.masked_fill(tril == 0, float('-inf')) # masking the upper triangular part
print(f"Weights before softmax: \n{weights}\n")
weights = F.softmax(weights, dim=1) # row normalization
print(f"Weights after softmax: \n{weights}")

Weights before softmax: 
tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

Weights after softmax: 
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250